In [ ]:
import zipfile
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Paths for the parent archive and where to unzip it
parent_zip_path = '/content/archive.zip'  # Path to the parent archive
unzipped_dir = '/content/unzipped_folder'  # Directory to unzip the archive to

# Unzip the parent archive containing 'train' and 'test' directories
with zipfile.ZipFile(parent_zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzipped_dir)

# Set the paths for the 'train' and 'test' directories (directly inside the unzipped folder)
train_dir = os.path.join(unzipped_dir, 'train')
test_dir = os.path.join(unzipped_dir, 'test')

# Use ImageDataGenerator to load and augment images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from the directories, split by class labels (emotion categories)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical',  # One-hot encoding for the labels
    color_mode='grayscale'  # Grayscale images
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical',
    color_mode='grayscale'  # Grayscale images
)

# Define the CNN model architecture
model = Sequential()

# Add Convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 classes for FER2013 emotions

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=50, validation_data=test_generator)

# Save the model
model.save('fer2013_emotion_recognition_model.h5')

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc * 100:.2f}%')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 126s 276ms/step - accuracy: 0.2371 - loss: 1.8323 - val_accuracy: 0.2732 - val_loss: 1.7578
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 120s 267ms/step - accuracy: 0.2890 - loss: 1.7474 - val_accuracy: 0.3728 - val_loss: 1.6355
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 119s 265ms/step - accuracy: 0.3457 - loss: 1.6738 - val_accuracy: 0.3944 - val_loss: 1.5801
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 120s 266ms/step - accuracy: 0.3599 - loss: 1.6417 - val_accuracy: 0.4207 - val_loss: 1.5289
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 143s 269ms/step - accuracy: 0.3875 - loss: 1.5885 - val_accuracy: 0.4259 - val_loss: 1.4963
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 119s 265ms/step - accuracy: 0.4039 - loss: 1.5537 - val_accuracy: 0.4387 - val_loss: 1.4732
Epoch 7/50
 53/449 ━━━━━━━━━━━━━━━━━━━━ 1:38 249ms/step - accuracy: 0.4025 - loss: 1.5397